In [3]:
import os

from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# API 키 가져오기
MY_OPENAI_KEY = os.getenv("MY_OPENAI_KEY")

client = OpenAI(api_key=MY_OPENAI_KEY)

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': 'hi'}],
    temperature=0.0
)

print(completion.choices[0].message.content)

Hello! How can I assist you today?


# 데이터 전처리 함수 작성

### 리뷰 JSON 파일 로드

In [9]:
import json

with open('./resources/reviews.json', 'r', encoding='utf-8') as f:
    review_list = json.load(f)

review_list[: 3]

[{'review': '서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍',
  'stars': 4,
  'date': '2024.01.24'},
 {'review': '인사동점 이용해보고 너무 좋아서 용산점 이용했습니다~\n건물도 너무 깨끗하고 직원들도 너무 친절했어요~\n게다가 무인락커가 있어서 락커에 짐보관이 너무 편하고좋았습니다~^^',
  'stars': 5,
  'date': '6일 전'},
 {'review': '친구가 생일이라서 다 같이 방문을 했는데 1

#### 좋은 평점 및 나쁜 평점 정의
 - 좋은 평점 = 별 5개
 - 나쁜 평점 = 별 4개 이하

In [10]:
good_count, bad_count = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_count += 1
    else:
        bad_count += 1

good_count, bad_count

(79, 21)

### 좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [11]:
reviews_good, reviews_bad = [], []
for r in review_list:
    if r['stars'] == 5:
        reviews_good.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가
    else:
        reviews_bad.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]') # 스페셜 토큰 추가

reviews_bad[:3]

['[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]',
 '[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네

In [13]:
reviews_good_text = '\n'.join(reviews_good)
reviews_bad_text = '\n'.join(reviews_bad)

reviews_bad_text[:1000]

'[REVIEW_START]서울 호캉스를 알아보다가 접근성 좋은 용산역 바로앞에 위치한 나인트리 프리미어 로카우스 호텔 서울 용산!! 정말 기대 이상으로 호텔시설, 애프터눈 티,\r\n수영장, 해피아워, 야경, 조식 이 모든거를 다 누렸는데!\r\n\r\n가격이 20만원 초중반이라니 😍 정말 엄마도 너무 좋아하시고 저도 너무 행복했던 야놀자에 있는 이그제큐티브 라운지 상품 강추합니다👍👍👍\r\n\r\n애프터눈티- 체크인 직후에 라운지에서 티타임❤️\r\n가볍게 샌드위치랑 쿠키가 나왔어요\r\n\r\n수영장- 1,2,3부 중에 원하는 시간 선택해서 이용하는 방식이고 그래서 그런지 다른 수영장 대비 사람이 엄청 많지도 않았어요 카바나 이런거도 무료라 무조건 빨리 입장하셔서 좋은자리 잡는거 추천드려용! 💕\r\n\r\n해피아워- 연어,불고기,치즈 각종음식들이 생각보다 종류도 많고 5성급 해피아워에 밀리지 않는 음식 퀼리티여서 진짜 해피아워 이용하면서 더 반한 로카우스!! 와인, 보드카가 좀 아쉬운거 빼고는 진짜 굿\r\n\r\n야경 정말 완벽 그자체! 제가 3번째 사진에서 찍은것은 해피아워 라운지10층 왼쪽복도 맨끝!!\r\n\r\n조식- 가짓수는 30~40여가지로 엄청 많은것도 적지도 않은 가짓수에요 근데 맛이 다 좋아서 ㅎㅎ\r\n아침부터 폭풍식사 했습니다. 다만 1부, 2부든 무조건 빨리가세오 안그러면 웨이팅 있습니다.\r\n\r\n나인트리 프리미어 로카우스 호텔 서울 용산\r\n서울 호캉스로 가성비도 좋고 기대이상 다 좋아서 만족도 정말 높은 호캉스였어용 또올게용❤️👍[REVIEW_END]\n[REVIEW_START]친구가 생일이라서 다 같이 방문을 했는데 1층에 카페가 있었는데 미니케이크와 숫자초까지 팔고 있어서 좋았고\n직원분들의 빠른 피드백과 체크인 절차, 자세한 설명이 좋았습니다 ! 수영장도 물 온도도 따뜻하게 이용할 수 있었고\n자쿠지도 있어서 마지막에 따뜻하게 이용할 수 있었습니다. 객실 내에 테이블도 있어서 밤에 야식도 시켜먹고 넘 좋았네요.\n

### 전처리 함수 작성

In [ ]:
def preprocess_reviews():
    pass